In [5]:
#####**************** put this file at the same file with .gxl files, and then run *********##########

import sys
from bs4 import BeautifulSoup as Soup

In [9]:
def convert(filename):
    #open graphgm file to read
    handler = open(filename).read()
    soup = Soup(handler, "xml")
    #create a new soup to write to new gexf file
    tagText = '''
                 <gexf version="1.1" xmlns="http://www.gexf.net/1.1draft" xmlns:viz="http://www.gexf.net/1.1draft/viz" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.w3.org/2001/XMLSchema-instance">
                  <graph defaultedgetype="undirected" mode="static">
                    <attributes class="node" mode="static">
                      <attribute id="0" title="node_class" type="string" />
                    </attributes>
                    <nodes>
                    </nodes>
                    <edges>
                    </edges>
                  </graph>
                </gexf>'''
    
    soup_result = Soup(tagText, "xml")
    #add start brackets to gexf
    #parse node data from graphml and put each node entry to gexf
    nodes_tag = soup_result.nodes
    for node in soup.findAll('node'):
        node_attrs = dict(node.attrs)
        node_id = int(node_attrs[u'id'])-1
        #print("node_id", node_id)
        node_label = 0
        aaLength_v = 0 #edge label
        sequence_v = ""
        for attr in node.findAll('attr'):
            if dict(attr.attrs)[u'name'] == 'type':
                node_label = (attr.findAll('int')[0]).text
            if dict(attr.attrs)[u'name'] == 'aaLength':
                aaLength_v = (attr.findAll('int')[0]).text
            if dict(attr.attrs)[u'name'] == 'sequence':
                sequence_v = (attr.findAll('int')[0]).text
        node_tag = soup_result.new_tag("node", id=node_id, label=node_id)
        attvalues_tag = soup_result.new_tag("attvalues")
        #attvalue_tag = Soup('<attvalue for="0" value="'+str(node_label)+'"/>', "xml")
        attvalue_tag = soup_result.new_tag("attvalue", value=node_label, aaLength=aaLength_v,sequence=sequence_v )
        attvalues_tag.append(attvalue_tag)
        node_tag.append(attvalues_tag)
        nodes_tag.append(node_tag)
    #parse edge data from graphml and put each edge entry to gexf
    edges_tag = soup_result.edges
    edge_index = 0
    for edge in soup.findAll('edge'):
        edge_attrs = dict(edge.attrs)
        start = int(edge_attrs[u'from'])-1
        end = int(edge_attrs[u'to'])-1
        frequency_v = 0
        type0_v = 0 #edge label
        distance0_v = 0
        for attr in edge.findAll('attr'):
            if dict(attr.attrs)[u'name'] == 'frequency':
                frequency_v = (attr.findAll('int')[0]).text
            if dict(attr.attrs)[u'name'] == 'type0':
                type0_v = (attr.findAll('double')[0]).text
            if dict(attr.attrs)[u'name'] == 'distance0':
                distance0_v = (attr.findAll('double')[0]).text
        edge_tag = soup_result.new_tag("edge", id=edge_index, source=start, target=end, 
                                       weight="1",frequency=frequency_v, type0=type0_v, distance0=distance0_v)
        edges_tag.append(edge_tag)
        edge_index += 1
    #add for attributes to each node
    attList = soup_result.findAll('attvalue')
    for att in attList:
        att.attrs['for'] = 0
    #save to gexf file
    sep = '.'
    rest = filename.split(sep, 1)[0]
    output_filename = rest+".gexf"
    with open('../enzymes_gexf_from_gxl/'+output_filename, 'w') as f:
        for line in soup_result.prettify():
            f.write(str(line))
    

In [10]:
import os

In [11]:
count = 0
for filename in os.listdir("."):
    if filename.endswith(".gxl"):
        convert(filename)
        count +=1
print(count)

600
